In [1]:
import project_path

import pandas as pd
from sof_sa.utility_functions import connect2db, db_config

creds = db_config("..\\dev_dbconfig.yaml", 0)
engine = connect2db(**creds)

In [5]:
# function to scrape data into staging db
import requests

url = 'https://www.globalbrainforce.com/blog/software-developer-salary-around-the-world/'

def scrape_data(url):
    """
    A function to scrape data from the web.
    Params :
        url - url for the web page to be scraped.
    Returns:
        Tables in the web page in a list of pandas dataframe objects
    """
    header = {
    "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/53>",
    "X-Requested-With" : "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)
    web_data = pd.read_html(r.text)
    return web_data

_data = scrape_data(url)[3]
_data

,Programming Language,Salary in USD
0,Perl,"$76,000"
1,Scala,"$76,000"
2,Go,"$74,000"
3,Rust,"$74,000"
4,Ruby,"$71,000"
5,Bash/Shell/PowerShell,"$65,000"
6,Objective-C,"$64,000"
7,Haskell,"$60,000"
8,Julia,"$59,000"
9,Python,"$59,000"


In [8]:
# Transformations
# Setting column names
_data.columns = ['progamming_language', 'salary_in_usd']

# Removing the $ sign from colmn salary_in_usd and setting type to float
_data['salary_in_usd'] = _data['salary_in_usd'].str.replace('$', '')
_data['salary_in_usd'] = _data['salary_in_usd'].astype(float)


Index(['progamming_language', 'salary_in_usd'], dtype='object')

In [ ]:
# Writing data to the database 
# Table name : programming_language_salary_staging
with engine.connect() as connection:
    _data.to_sql('programming_language_salary_staging', con=connection, if_exists='replace')

In [10]:
# Now working with csv files from stackoverflow survey
# loading in data 
raw_df20 = pd.read_csv("..\\data\\survey_results_public_2020.csv")
raw_df21 = pd.read_csv("..\\data\\survey_results_public_2021.csv")

with engine.connect() as connection:
    raw_df20.to_sql('raw_2020', con=connection, if_exists='replace')
    raw_df21.to_sql('raw_2021', con=connection, if_exists='replace')


,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [4]:
raw_df21 = pd.read_csv("..\\data\\2020_sharing_data_outside.csv")
raw_df21.head()

C:\Users\nzman\AppData\Local\Temp\ipykernel_16732\4032837907.py:1: DtypeWarning: Columns (8,14,15,22,23,42,43,48,51,52,53,54,56,66,74,78,87,92,100,101,102,109,110,121,122,123,126,127,129,130,131,133,134,135,136,137,138,139,140,142,143,144,146,148,152,154,155,156,159,160,161,163,164,165,166,169,170,171,172,173,175,176,177,178,179,180,181,182,215,221,231,232,236,239,240,243,244,245,246,247,248,251,252,254,257,258,259,260,271,272,279,286,289,290,291,293,304,307,308,310,312,313,315,316,317,320,321,324,326,329,330,336,337,340,341,342,347,348,349,350,351,355,356,357,358,359,360,362,363,366,367,368,369,371,372,376,377,378,379,380,382,383,384,385,386,387,388,396,410,412,414,417,420,421,422,423,424,425,426,427,428,429,432,433,434,444,449,452,454,455,457,458,459,463,465,466,467,468,469,470,473,474,475,477,480,481,482,484,487,488,489,490,493,494,495,496,497,499,500,501,502,503,504,505,507,510,511,515,516,517,518,521,522,523,524,525,526,527,528,530,531,532,534,536,538,539,540,541,544,545,548,550,5

,employment_status,job_role.Developer / Programmer / Software Engineer,job_role.DevOps Engineer / Infrastructure Developer,job_role.DBA,job_role.Architect,job_role.Tester / QA Engineer,job_role.Technical Support,job_role.Data Analyst / Data Engineer/ Data Scientist,job_role.Business Analyst,job_role.Team Lead,...,primary.F#,rank.Assembly,rank.Julia,primary.Perl,rank.I don't use programming languages,primary.I don't use programming languages,is_employed,main,source,weight
0,Fully employed by a company / organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Team Lead,...,NaN,NaN,NaN,NaN,NaN,NaN,Employed,JavaScript,external,0.653323
1,Fully employed by a company / organization,NaN,NaN,NaN,Architect,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Employed,Java,external,0.797550
2,Freelancer (a person pursuing a profession wit...,Developer / Programmer / Software Engineer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Employed,Java,external,0.302700
3,Fully employed by a company / organization,Developer / Programmer / Software Engineer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Team Lead,...,NaN,NaN,NaN,NaN,NaN,NaN,Employed,Java,internal,0.177387
4,Fully employed by a company / organization,Developer / Programmer / Software Engineer,DevOps Engineer / Infrastructure Developer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Employed,Java,internal,0.000000
